<a href="https://colab.research.google.com/github/PacktPublishing/Machine-Learning-for-Time-Series-with-Python/blob/master/chapter10/Causal_CNN.ipynb" target="_parent\"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Conv1D, Input, Add, Activation, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import optimizers

# Callback pour arrêt anticipé
callback = EarlyStopping(monitor='loss', patience=10)

# Bloc DC-CNN
def DC_CNN_Block(nb_filter, filter_length, dilation):
    def f(input_):
        residual = input_
        layer_out = Conv1D(
            filters=nb_filter, kernel_size=filter_length,
            dilation_rate=dilation,
            activation='linear', padding='causal', use_bias=False
        )(input_)
        layer_out = Activation('selu')(layer_out)
        skip_out = Conv1D(1, 1, activation='linear', use_bias=False)(layer_out)
        network_in = Conv1D(1, 1, activation='linear', use_bias=False)(layer_out)
        network_out = Add()([residual, network_in])
        return network_out, skip_out
    return f

# Modèle DC-CNN complet
def DC_CNN_Model(length):
    input = Input(shape=(length, 1))
    l1a, l1b = DC_CNN_Block(32, 2, 1)(input)
    l2a, l2b = DC_CNN_Block(32, 2, 2)(l1a)
    l3a, l3b = DC_CNN_Block(32, 2, 4)(l2a)
    l4a, l4b = DC_CNN_Block(32, 2, 8)(l3a)
    l5a, l5b = DC_CNN_Block(32, 2, 16)(l4a)
    l6a, l6b = DC_CNN_Block(32, 2, 32)(l5a)
    l6b = Dropout(0.8)(l6b)
    l7a, l7b = DC_CNN_Block(32, 2, 64)(l6a)
    l7b = Dropout(0.8)(l7b)
    l8 = Add()([l1b, l2b, l3b, l4b, l5b, l6b, l7b])
    l9 = Activation('relu')(l8)
    l21 = Conv1D(1, 1, activation='linear', use_bias=False)(l9)
    model = Model(inputs=input, outputs=l21)
    model.compile(loss='mae', optimizer=optimizers.Adam(), metrics=['mse'])
    return model


2025-05-30 12:33:53.869495: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-30 12:33:53.879113: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748601233.889154   29010 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748601233.892035   29010 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748601233.899742   29010 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:
def fit_model(timeseries):
    length = len(timeseries)-1
    model = DC_CNN_Model(length)
    model.summary()
    X = timeseries[:-1].reshape(1,length, 1)
    y = timeseries[1:].reshape(1,length, 1)
    model.fit(X, y, epochs=3000, callbacks=[callback])
    return model
    
def forecast(model, timeseries, horizon: int):
    length = len(timeseries)-1
    pred_array = np.zeros(horizon).reshape(1, horizon, 1)
    X_test_initial = timeseries[1:].reshape(1,length,1)
    
    pred_array[: ,0, :] = model.predict(X_test_initial)[:, -1:, :]
    for i in range(horizon-1):
        pred_array[:, i+1:, :] = model.predict(
            np.append(
                X_test_initial[:, i+1:, :], 
                pred_array[:, :i+1, :]
            ).reshape(1, length, 1))[:, -1:, :]
    return pred_array.flatten()
    
def evaluate_timeseries(series, horizon: int):
    model = fit_model(series)
    pred_array = forecast(model, series, horizon)
    return pred_array, model

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def show_result(y_test, predicted, ylabel="Passengers"):
  plt.figure(figsize=(16, 6))
  plt.plot(y_test.index, predicted, 'o-', label="predicted")
  plt.plot(y_test.index, y_test, '.-', label="actual")

  plt.ylabel(ylabel)
  plt.legend()

In [4]:
import pandas as pd

values = [         
  112., 118., 132., 129., 121., 135., 148., 148., 136., 119., 104., 118., 115., 126.,
  141., 135., 125., 149., 170., 170., 158., 133., 114., 140., 145., 150., 178., 163.,
  172., 178., 199., 199., 184., 162., 146., 166., 171., 180., 193., 181., 183., 218.,
  230., 242., 209., 191., 172., 194., 196., 196., 236., 235., 229., 243., 264., 272.,
  237., 211., 180., 201., 204., 188., 235., 227., 234., 264., 302., 293., 259., 229.,
  203., 229., 242., 233., 267., 269., 270., 315., 364., 347., 312., 274., 237., 278.,
  284., 277., 317., 313., 318., 374., 413., 405., 355., 306., 271., 306., 315., 301.,
  356., 348., 355., 422., 465., 467., 404., 347., 305., 336., 340., 318., 362., 348.,
  363., 435., 491., 505., 404., 359., 310., 337., 360., 342., 406., 396., 420., 472.,
  548., 559., 463., 407., 362., 405., 417., 391., 419., 461., 472., 535., 622., 606.,
  508., 461., 390., 432.,
 ]
idx = pd.date_range("1949-01-01", periods=len(values), freq="M")
passengers = pd.Series(values, index=idx, name="passengers").to_frame()

/tmp/ipykernel_29010/3813778322.py:16: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  idx = pd.date_range("1949-01-01", periods=len(values), freq="M")


In [ ]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(
    passengers.passengers, passengers.passengers.shift(-1), shuffle=False
)
HORIZON = len(y_test)
predictions, model = evaluate_timeseries(X_train.values.reshape(-1, 1), horizon=HORIZON)

W0000 00:00:1748601235.084173   29010 gpu_device.cc:2341] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 107, 1)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, 107, 32)   │         64 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 107, 32)   │          0 │ conv1d[0][0]      │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_2 (Conv1D)   │ (None, 107, 1)    │         32 │ activation[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 107, 1)    │          0 │ input_layer[0][0… │
│                     │                   │            │ conv1d_2[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_3 (Conv1D)   │ (None, 107, 32)   │         64 │ add[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 107, 32)   │          0 │ conv1d_3[0][0]    │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_5 (Conv1D)   │ (None, 107, 1)    │         32 │ activation_1[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 107, 1)    │          0 │ add[0][0],        │
│                     │                   │            │ conv1d_5[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_6 (Conv1D)   │ (None, 107, 32)   │         64 │ add_1[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 107, 32)   │          0 │ conv1d_6[0][0]    │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_8 (Conv1D)   │ (None, 107, 1)    │         32 │ activation_2[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_2 (Add)         │ (None, 107, 1)    │          0 │ add_1[0][0],      │
│                     │                   │            │ conv1d_8[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_9 (Conv1D)   │ (None, 107, 32)   │         64 │ add_2[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 107, 32)   │          0 │ conv1d_9[0][0]    │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_11 (Conv1D)  │ (None, 107, 1)    │         32 │ activation_3[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_3 (Add)         │ (None, 107, 1)    │          0 │ add_2[0][0],      │
│                     │                   │            │ conv1d_11[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_12 (Conv1D)  │ (None, 107, 32)   │         64 │ add_3[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_4        │ (None, 107, 32)   │          0 │ conv1d_12[0][0]   │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_14 (Conv1D)  │ (None, 107, 1)    │         32 │ activation_4[0][

 Total params: 865 (3.38 KB)

 Trainable params: 865 (3.38 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/3000
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 234.0966 - mse: 61826.9531
Epoch 2/3000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 232.5048 - mse: 61146.1367
Epoch 3/3000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 232.3938 - mse: 61109.7305
Epoch 4/3000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: 232.1822 - mse: 61057.8867
Epoch 5/3000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 232.0591 - mse: 61029.7070
Epoch 6/3000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 232.0183 - mse: 61019.5664
Epoch 7/3000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: 232.0093 - mse: 61017.6914
Epoch 8/3000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 232.0093 - mse: 61017.6914
Epoch 9/3000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 232.0093 - mse: 61017.6914
Epoch 10/3000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 232.0093 - mse: 61017.6914
Epoch 11/3000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 232.0093 - mse: 61017.6914
Epoch 12/3000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - 

In [14]:
show_result(y_test[:HORIZON], predictions[:HORIZON], "Passengers")